# Partie 2 : Comprendre les Outils et les Agents

Ce notebook couvre deux concepts fondamentaux :
1. **Les Outils (Function Calling)** : Comment les LLMs utilisent des fonctions externes
2. **LangGraph** : Un framework pour créer des agents sous forme de graphes d'états

---

## Chapitre 1 : Les Outils (Function Calling)

### Qu'est-ce qu'un Outil ?

Les outils permettent aux LLMs de :
- Effectuer des calculs
- Interroger des bases de données
- Appeler des APIs externes

**Concept clé** : Les LLMs n'exécutent pas directement les outils. Ils **génèrent des instructions structurées** qui nous indiquent quel outil appeler et avec quels paramètres.

### Configuration

In [ ]:
import json
from openai import AzureOpenAI

config = {
    "openai_endpoint": "",
    "openai_key": "",
    "chat_deployment": ""
}

client = AzureOpenAI(
    api_key=config["openai_key"],
    api_version="2024-02-01",
    azure_endpoint=config["openai_endpoint"]
)

print("✅ Client Azure OpenAI initialisé")

### Exemple : Une Calculatrice Simple

In [ ]:
def calculatrice(operation: str, a: float, b: float) -> float:
    """Une calculatrice simple qui effectue des opérations de base."""
    if operation == "additionner":
        return a + b
    elif operation == "soustraire":
        return a - b
    elif operation == "multiplier":
        return a * b
    elif operation == "diviser":
        if b == 0:
            return "Erreur : Division par zéro"
        return a / b
    else:
        return f"Erreur : Opération inconnue '{operation}'"

print(f"Test: 25 × 4 = {calculatrice('multiplier', 25, 4)}")

### Pourquoi un LLM a-t-il besoin d'une calculatrice ?

Testons le LLM sur un calcul complexe :

In [ ]:
prompt = "multiplie 9712439 et -138213"  # Bonne réponse: -1342385331507

response = client.chat.completions.create(
    model=config["chat_deployment"],
    messages=[{"role": "user", "content": prompt}]
)

print("RÉPONSE DU LLM (sans outil) :")
print(response.choices[0].message.content)

**Questions :**
1. La réponse est-elle correcte ? Essayez plusieurs fois.
2. Comment pourrait-on permettre au LLM d'utiliser notre calculatrice ?

---
## Approche Manuelle

### Étape 1 : Décrire l'outil dans le prompt

In [ ]:
description_outil = """**calculatrice** : Effectue des opérations arithmétiques
Paramètres :
- operation: "additionner", "soustraire", "multiplier", ou "diviser"
- a: le premier nombre
- b: le deuxième nombre

Pour utiliser un outil, réponds avec CE FORMAT JSON EXACT :
```json
{"outil": "calculatrice", "arguments": {"operation": "multiplier", "a": 25, "b": 4}}
```"""

message_utilisateur = "Combien font 9712439 multiplié par -138213 ?"

prompt = f"""Tu es un assistant avec accès à des outils.

{description_outil}

Question : {message_utilisateur}"""

response = client.chat.completions.create(
    model=config["chat_deployment"],
    messages=[{"role": "user", "content": prompt}]
)

reponse_texte = response.choices[0].message.content
print("RÉPONSE DU LLM :")
print(reponse_texte)

### Étape 2 : Parser le JSON

In [ ]:
import re

try:
    json_match = re.search(r'```json\s*(\{.*?\})\s*```', reponse_texte, re.DOTALL)
    
    if json_match:
        json_str = json_match.group(1)
    else:
        json_str = reponse_texte.strip()
    
    appel_outil = json.loads(json_str)
    nom_outil = appel_outil.get("outil")
    arguments = appel_outil.get("arguments")
    
    print(f"✅ Outil : {nom_outil}")
    print(f"✅ Arguments : {arguments}")
    
except Exception as e:
    print(f"❌ Erreur de parsing : {e}")

### Étape 3 : Exécuter l'outil

In [ ]:
# Bonne réponse: -1342385331507
if nom_outil == "calculatrice":
    resultat = calculatrice(**arguments)
    print(f"✅ Résultat : {resultat}")

**Questions :**
1. La réponse est-elle correcte maintenant ?
2. Quels sont les problèmes de cette approche manuelle ?

---
## Approche API (tools=)

Les APIs de LLM intègrent une notion d'outils pour faciliter les interactions.

### Schéma de l'outil

In [ ]:
outil_calculatrice = {
    "type": "function",
    "function": {
        "name": "calculatrice",
        "description": "Effectue des opérations arithmétiques (additionner, soustraire, multiplier, diviser).",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "enum": ["additionner", "soustraire", "multiplier", "diviser"],
                    "description": "L'opération à effectuer"
                },
                "a": {"type": "number", "description": "Le premier nombre"},
                "b": {"type": "number", "description": "Le deuxième nombre"}
            },
            "required": ["operation", "a", "b"]
        }
    }
}

print(json.dumps(outil_calculatrice, indent=2, ensure_ascii=False))

### Appel avec tools=

In [ ]:
response = client.chat.completions.create(
    model=config["chat_deployment"],
    messages=[{"role": "user", "content": "Combien font 9712439 multiplié par -138213 ?"}],
    tools=[outil_calculatrice],
    tool_choice="auto"
)

message = response.choices[0].message

if message.tool_calls:
    for tool_call in message.tool_calls:
        print(f"✅ Outil : {tool_call.function.name}")
        print(f"✅ Arguments : {tool_call.function.arguments}")
        
        arguments = json.loads(tool_call.function.arguments)
        resultat = calculatrice(**arguments)
        print(f"✅ Résultat : {resultat}")

### Boucle Complète : LLM → Outil → LLM

In [ ]:
def executer_outil(nom_outil: str, arguments: dict):
    """Exécute l'outil demandé."""
    if nom_outil == "calculatrice":
        return calculatrice(**arguments)
    return f"Erreur : Outil inconnu '{nom_outil}'"

def boucle_agent(message_utilisateur: str, outils: list):
    """Boucle complète : LLM → Outil → Réponse Finale."""
    print(f"👤 Utilisateur : {message_utilisateur}\n")
    
    messages = [{"role": "user", "content": message_utilisateur}]
    
    response = client.chat.completions.create(
        model=config["chat_deployment"],
        messages=messages,
        tools=outils,
        tool_choice="auto"
    )
    
    message = response.choices[0].message
    
    if not message.tool_calls:
        print(f"🤖 Réponse directe : {message.content}")
        return
    
    messages.append(message)
    
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        print(f"🔧 Outil : {tool_call.function.name}({arguments})")
        
        resultat = executer_outil(tool_call.function.name, arguments)
        print(f"📊 Résultat : {resultat}\n")
        
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_call.function.name,
            "content": str(resultat)
        })
    
    reponse_finale = client.chat.completions.create(
        model=config["chat_deployment"],
        messages=messages
    )
    
    print(f"🤖 Réponse : {reponse_finale.choices[0].message.content}")

# Test
boucle_agent("Combien font 1547 divisé par 23 ?", [outil_calculatrice])

In [ ]:
# Tests supplémentaires
questions = [
    "Combien font 892 multiplié par 47 ?",
    "Si j'ai 1000 euros et que je dépense 347, combien me reste-t-il ?",
]

for q in questions:
    print("=" * 60)
    boucle_agent(q, [outil_calculatrice])
    print()

---
## Pièges Courants

### Piège 1 : Descriptions vagues

In [ ]:
# MAUVAIS : Description vague
mauvais_outil = {
    "type": "function",
    "function": {
        "name": "outil",
        "description": "outil",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {"type": "string"},
                "a": {"type": "number"},
                "b": {"type": "number"}
            },
            "required": ["operation", "a", "b"]
        }
    }
}

print("Test avec une MAUVAISE description :")
boucle_agent("Combien font 25 fois 4 ?", [mauvais_outil])

### Piège 2 : Gestion des erreurs

In [ ]:
print("Test de la division par zéro :")
boucle_agent("Combien font 100 divisé par 0 ?", [outil_calculatrice])

---
## Résumé : Flux d'Appel d'Outils

```
Question Utilisateur
        ↓
   LLM (avec outils)
        ↓
   tool_calls ? ──Non──→ Réponse directe
        │
       Oui
        ↓
   Exécution Outil
        ↓
   Résultat → LLM
        ↓
   Réponse Finale
```

---
# Chapitre 2 : Introduction à LangGraph

## Qu'est-ce que LangGraph ?

**LangGraph** permet de créer des agents LLM sous forme de **graphes d'états**.

Au lieu de gérer manuellement la boucle, LangGraph offre :
- Une **abstraction claire** : Nœuds et arêtes représentent le flux
- Une **gestion automatique** : La boucle agent → outil → agent est gérée automatiquement
- Une **extensibilité** : Facile d'ajouter de nouveaux comportements

### Configuration LangGraph

In [ ]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
from typing import Literal

print("✅ Imports LangGraph réussis")

### Définir l'outil avec le décorateur @tool

Le décorateur `@tool` génère automatiquement le schéma JSON à partir :
- Des types Python (str, float → string, number)
- De `Literal` pour les enums
- De la docstring pour les descriptions

In [ ]:
@tool
def calculatrice(
    operation: Literal["additionner", "soustraire", "multiplier", "diviser"],
    a: float,
    b: float
) -> float:
    """Effectue des opérations arithmétiques de base.
    
    Args:
        operation: L'opération à effectuer
        a: Le premier nombre
        b: Le deuxième nombre
    """
    if operation == "additionner":
        return a + b
    elif operation == "soustraire":
        return a - b
    elif operation == "multiplier":
        return a * b
    elif operation == "diviser":
        return a / b if b != 0 else "Erreur : Division par zéro"

# Test
print(f"Test: 25 × 4 = {calculatrice.invoke({'operation': 'multiplier', 'a': 25, 'b': 4})}")

In [ ]:
# Voir le schéma généré automatiquement
print("Schéma généré par @tool :")
print(json.dumps(calculatrice.args_schema.model_json_schema(), indent=2, ensure_ascii=False))

### Créer le LLM avec les outils

In [ ]:
llm = AzureChatOpenAI(
    api_key=config["openai_key"],
    api_version="2024-02-01",
    azure_endpoint=config["openai_endpoint"],
    model=config["chat_deployment"],
    temperature=0
)

tools = [calculatrice]
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM configuré avec les outils")

### Construire le Graphe

Un graphe LangGraph se compose de :
1. **Nœuds** : Fonctions qui traitent l'état
2. **Arêtes** : Transitions entre nœuds
3. **État** : Données qui circulent (ici, les messages)

In [ ]:
# Nœud Agent : appelle le LLM
def call_agent(state: MessagesState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# Nœud Outils : exécute les outils demandés
tool_node = ToolNode(tools)

# Routage : continuer vers les outils ou terminer ?
def should_continue(state: MessagesState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

print("✅ Nœuds et routage définis")

In [ ]:
# Construire le graphe
workflow = StateGraph(MessagesState)

# Ajouter les nœuds
workflow.add_node("agent", call_agent)
workflow.add_node("tools", tool_node)

# Définir les arêtes
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "agent")

# Compiler
app = workflow.compile()

print("✅ Graphe compilé")
print("\nStructure : START → agent → [tools → agent]* → END")

In [ ]:
# Visualiser le graphe
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except:
    print("Visualisation non disponible")

### Tester l'Agent

In [ ]:
question = "Combien font 9712439 multiplié par -138213 ?"

result = app.invoke({"messages": [HumanMessage(content=question)]})

print(f"❓ Question : {question}")
print(f"🤖 Réponse : {result['messages'][-1].content}")

### Observer le Flux avec Streaming

In [ ]:
question = "Si je multiplie 847 par 123, puis divise par 11, quel est le résultat ?"

print(f"❓ Question : {question}\n")
print("FLUX D'EXÉCUTION :")
print("=" * 50)

for i, step in enumerate(app.stream({"messages": [HumanMessage(content=question)]}), 1):
    for node_name, node_state in step.items():
        print(f"\nÉtape {i} - {node_name}")
        
        if node_name == "agent":
            last_msg = node_state["messages"][-1]
            if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                for tc in last_msg.tool_calls:
                    print(f"   → Appel : {tc['name']}({tc['args']})")
            else:
                print(f"   → Réponse finale prête")
        
        elif node_name == "tools":
            for msg in node_state["messages"]:
                print(f"   → Résultat : {msg.content}")

print("\n" + "=" * 50)
print(f"🤖 Réponse : {list(step.values())[0]['messages'][-1].content}")

---
## Résumé

### Ce que nous avons appris :

1. **Function Calling** : Les LLMs génèrent des instructions structurées pour appeler des outils
2. **Schéma d'outil** : Description JSON qui permet au LLM de comprendre l'outil
3. **Boucle Agent** : LLM → Outil → LLM → Réponse
4. **LangGraph** : Framework qui automatise cette boucle avec des graphes d'états

### Prochaines étapes :

Dans les prochains notebooks, nous verrons des cas d'usage concrets avec des outils RAG, SQL, et API.